# Chapter 8

## 8.1 Models and Planning

## 8.2 Dyna: Integrated Planning, Acting and Learning

In [40]:
(6, 2) in 𝓢

false

In [45]:
𝓐 = [ :left, :up, :right, :down ]
𝓢 = setdiff([ (i, j) for i in 1:9, j in 1:6 ], [(3, i) for i in 3:5], [(8, i) for i in 4:6], [(6, 2)])
γ = 0.95

function g(s, a)
    if a == :left
        sp = (clamp(s[1] - 1, 1, 9), s[2])
    elseif a == :up
        sp = (s[1], clamp(s[2] + 1, 1, 6))
    elseif a == :right
        sp = (clamp(s[1] + 1, 1, 9), s[2])
    elseif a == :down
        sp = (s[1], clamp(s[2] - 1, 1, 6))
    else
        error("Not a valid action")
    end
    sp = sp in 𝓢 ? sp : s
    r = sp == (9, 6) ? 1 : 0
    return sp, r
end

g (generic function with 1 method)

In [ ]:
function dyna_q(episodes, n; ε=0.1, α=0.1)
    Q = Dict((s, a) => 0.0 for s in 𝓢, a in 𝓐)
    model = Dict((s, a) => 0.0, s for s in 𝓢, a in 𝓐)
    steps = Int64[]
    for episode in 1:episodes
        s = rand(setdiff(𝓢, (9,6)))
        a = rand() < ε ? rand(𝓐) : argmax(a -> Q[(s, a)], 𝓐)
        sp, r = g(s, a)
        Q[(s, a)] = Q[(s, a)] + α*(r + γ*max(a -> Q[(s, a)], 𝓐) - Q[(s, a)])
        model[(s, a)] = r, sp
        for i in 1:n
            
        end
    end
end

## 8.3 When the Model Is Wrong